# Ciencia de Datos - TP6

## Integrantes

- Ambroa, Nicolás - 229/13 - ambroanicolas@hotmail.com
- Gaustein, Diego - 586/09 - diego@gaustein.com.ar

In [58]:
from collections import Counter
from functools import wraps
from itertools import chain
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
import os
import pickle
from pprint import pprint
from lxml import html
from nltk import sent_tokenize, word_tokenize, pos_tag, WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import sent_tokenize
from sklearn.metrics import mutual_info_score
import gensim

lemmatizer = WordNetLemmatizer()
stopwords = frozenset(stopwords.words('english'))

def load_or_call(func):
    """ Decorador auxiliar para cachear resultados en un .pickle. """
    @wraps(func)
    def wrapper(*args, **kwargs):
        filename = '{}-{}-{}.pickle'.format(func.__name__, str(args), str(kwargs))
        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                return pickle.load(f)
        else:
            res = func(*args, **kwargs)
            with open(filename, 'wb') as f:
                pickle.dump(res, f)
            return res

    return wrapper

### 1.1. Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario `(<DOCNO>: <TEXT>)`.

In [2]:
parsed = html.parse('ap/ap.txt')
documents = {}
for document in parsed.iter('doc'):
    docno, article = document.getchildren()
    documents[docno.text.strip()] = article.text.strip()

print('Cargados', len(documents), 'articulos')


Cargados 2250 articulos


### 1.2. Calcular el tamaño del vocabulario.

In [3]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN  # El default es NOUN

def get_words_for_document(document):
    for sentence in sent_tokenize(document):
        tagged_sentence = pos_tag(sentence.split())
        for word, pos in tagged_sentence:
            # Lematizar
            word = lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos))
            # Strip punctuation
            word = word.strip(punctuation)
            # Lowercase
            word = word.lower()
            
            # Skip stopwords and punctuation
            if len(word) > 1 and word not in stopwords:
                yield word

@load_or_call
def get_word_count():
    c = Counter()
    for document in documents.values():
        c.update(get_words_for_document(document))
    return c
        
c = get_word_count()
cantidad_de_palabras = sum(c.values())
print(cantidad_de_palabras, 'palabras,', len(c), 'palabras distintas.')

548169 palabras, 41090 palabras distintas.


### 1.3. Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada.

In [4]:
"""
Para cada palabra, aproximamos p(palabra) con su frecuencia relativa según el counter.
Tomamos las 500 con más apariciones y recorremos el texto. Cada vez que encontremos una de estas miramos en una
ventana (n=8) cuando aparece alguna de las otras, y le sumamos uno.
Esta es nuestra estimación de p(palabra1, palabra2). Finalmente calculamos la información mutua e imprimimos los
pares más asociados.
"""

def mirar_ventana_y_buscar(palabra_a_asociar, palabras, palabras_a_buscar, indice_inicial, longitud_de_ventana, asociaciones):
    # Arreglo los índices para no pasarme en caso de una ventana muy cercana a los extremos de la lista.
    if indice_inicial - longitud_de_ventana < 0:
        indice_inicial = 0
    else:
        indice_inicial -= longitud_de_ventana
    if indice_inicial + longitud_de_ventana > len(palabras):
        indice_final = len(palabras)
    else:
        indice_final = indice_inicial + longitud_de_ventana
       
    # Recorro los indices, buscando palabras a buscar que no sean la palabra a asociar.
    for indice in range(indice_inicial, indice_final):
        if palabras[indice] in palabras_a_buscar and palabras[indice] != palabra_a_asociar:
            subindice_dict = str(palabra_a_asociar) + " " +str(palabras[indice])
            # Si la encontre, aumento en uno la asociación entre ambas palabras.
            try:
                asociaciones[subindice_dict] += 1
            except IndexError:
                asociaciones[subindice_dict] = 1

@load_or_call
def contar_asociaciones_de_palabras():
    contador_asociaciones_de_palabras = Counter()
    # Recorremos el texto.
    for document in documents.values():
        palabras = document.split()
        for indice, palabra in enumerate(palabras):
            #Cada vez que encontremos una de ellas, miramos en una ventana de n=8.
            if palabra in palabras_con_mas_apariciones:
                mirar_ventana_y_buscar(palabra, palabras, palabras_con_mas_apariciones, indice, 8, contador_asociaciones_de_palabras)
    return contador_asociaciones_de_palabras

# Tomamos las 500 palabras con más apariciones.
palabras_y_apariciones = c.most_common(500)
palabras_con_mas_apariciones = [tupla[0] for tupla in palabras_y_apariciones]

asociaciones_de_palabras = contar_asociaciones_de_palabras()
suma_de_asociaciones_totales = sum(asociaciones_de_palabras.values())
informaciones_mutuas = Counter()
# Recorremos los pares y calculamos información mutua para cada uno.
for palabras, asociaciones in asociaciones_de_palabras.items():
    palabras_parseadas = palabras.split(" ")
    palabra_x = palabras_parseadas[0]
    palabra_y = palabras_parseadas[1]
    proba_x = c[palabra_x] / float(cantidad_de_palabras)
    proba_y = c[palabra_y] / float(cantidad_de_palabras)
    proba_conj_x_y  = asociaciones_de_palabras[palabras] / suma_de_asociaciones_totales
    inf_mutua_x_y = proba_conj_x_y / (proba_x*proba_y)
    informaciones_mutuas[palabras] = inf_mutua_x_y

# Finalmente imprimimos los 5 pares más asociados según la información mutua
print("Los 5 pares de palabras más asociadas según la medida de información mutua son: \n")
for palabra_y_info_mutua in informaciones_mutuas.most_common(5):
    print("* {0} con información mutua: {1}".format(palabra_y_info_mutua[0], palabra_y_info_mutua[1]))

Los 5 pares de palabras más asociadas según la medida de información mutua son: 

* index value con información mutua: 1838.0320849868358
* conference news con información mutua: 1421.3935232619224
* 30 average con información mutua: 1384.3817325708626
* executive chief con información mutua: 1228.8735910193982
* fell index con información mutua: 1106.339421879901


## 3) Word embeddings, distancia semántica y Word-Net

### a) Utilizando el test WordSim353 , comparar el rendimiento entre LSA y Word2Vec.

#### Modelo Word2Vec

In [ ]:
# Docs para usar word2vec
# https://radimrehurek.com/gensim/models/word2vec.html
# https://rare-technologies.com/word2vec-tutorial/
# Para el model de Google: http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/

@load_or_call
def obtener_oraciones_de_corpus():
    oraciones = []
    for texto in documents.values():
        texto_por_oraciones = sent_tokenize(texto)
        oraciones.append(texto_por_oraciones)
    return oraciones

oraciones = obtener_oraciones_de_corpus()
word2vec_model = model = gensim.models.KeyedVectors.load_word2vec_format('/word2vec_model/GoogleNews-vectors-negative300.bin', binary=True)  
word2vec_model = gensim.models.Word2Vec(oraciones, iter=1, workers=4)

#### Modelo LSA

In [61]:
# Buscar corpus en http://www.nltk.org/data.html creo que le brown corpus sirve.
# Para instalar los corpus: sudo python -m nltk.downloader -d /usr/local/share/nltk_data all

lsa_corpus = 'nombre_del_archivo_donde_esta_el_corpus'
#Document-Term Matrix
cv = CountVectorizer(input='filename',strip_accents='ascii')
dtMatrix = cv.fit_transform(lsa_corpus).toarray()
print (dtMatrix.shape)
featurenames = cv.get_feature_names()
print (featurenames)

#Tf-idf Transformation
tfidf = TfidfTransformer()
tfidfMatrix = tfidf.fit_transform(dtMatrix).toarray()
print (tfidfMatrix.shape)

#SVD
#n_components is recommended to be 100 by Sklearn Documentation for LSA
#http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
svd = TruncatedSVD(n_components = 100)
svdMatrix = svd.fit_transform(tfidfMatrix)

print (svdMatrix)

#Cosine-Similarity
#cosine = cosine_similarity(svdMatrix[1], svdMatrix)

AttributeError: 'list' object has no attribute 'lower'

#### Testeo Similitud y Relatedness

In [ ]:
# Acá comparo las similitudes entre las palabras de wordsimtest test similitud
# EJ: model.similarity('woman', 'man')
similarities = {'WordSim353': {}, 'Word2Vec': {}, 'LSA': {}}
similarity_test_filename = 'wordsimtest/wordsim_similarity_goldstandard.txt'
with open(similarity_test_filename, 'rb') as similarity_test_file:
    for line in similarity_test_file.readlines():
        decoded_line = line.decode('UTF-8').split()
        similiarity_from_test = decoded_line[2]
        first_word = decoded_line[0]
        second_word = decoded_line[1]
        subindex = first_word + " " + second_word
        similarity['WordSim353'][subindex] = similiarity_from_test
        # ToDo: agregar la parte de LSA
        # De word2vec falta hacer:
        # word2vec_similarity = word2vec_model.similarity(first_word, second_word)
        # similarities['Word2Vec'][subindex] = word2vec_similarity
        
# ToDo: Lo mismo de arriba pero para relatedness?